# Homework

Below is the source code for the homework in [Chapter 3 - Working with sequences in TensorFlow](./ch03-working-with-sequences-in-tensorflow.ipynb).

After executing the code, start TensorBoard with `tensorboard --logdir /tmp/tensorflow-workshop/alphabet-predictor`. Once you navigate to `localhost:6006` you should see something similar to the following:
- In the `Scalars` tab
  ![TensorBoard scalars](./img/hw02-scalars.png)
- In the `Graphs` tab, after removing `train` and `rnn` nodes from the main graph:
  ![Computational graph](./img/hw02-graph.png)

In [ ]:
import os.path as path
import datetime
import numpy as np
import tensorflow as tf
from tensorflow.contrib import rnn
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

np.random.seed(2018)

alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

class Encoding():
    def __init__(self):
        self._char_to_int = dict((c, i) for i, c in enumerate(alphabet))
        self._int_to_char = dict((i, c) for i, c in enumerate(alphabet))

    def encode_sequence(self, sequence):
        return [self._char_to_int[char] for char in sequence]

    def encode_letter(self, letter):
        return self._char_to_int[letter]

    def decode_letter(self, value):
        return self._int_to_char[value]

class Dataset():
    def __init__(self, size=1000, seq_length=5, print_data=True):
        self._size = size
        self._sequence_length = seq_length
        self._inputs = []
        self._labels = []
        self._print_data = print_data
        self._encoding = Encoding()

    @property
    def inputs(self):
        return self._inputs

    @property
    def labels(self):
        return self._labels

    @property
    def num_classes(self):
        return len(alphabet)

    @property
    def sequence_length(self):
        return self._sequence_length

    def initialize(self):
        self._generate_random_data()
        self._reshape_inputs()
        self._normalize_inputs()
        self._normalize_labels()

    def shuffle(self):
        perm = np.arange(self._size)
        np.random.shuffle(perm)
        self._inputs = self._inputs[perm]
        self._labels = self._labels[perm]


    def _normalize_labels(self):
        self._labels = to_categorical(self._labels, num_classes=self.num_classes)
        self._labels = np.reshape(self._labels, (self._size, 1, self.num_classes))

    def _reshape_inputs(self):
        self._inputs = pad_sequences(self._inputs,
                                     maxlen=self._sequence_length,
                                     dtype='float32')
        self._inputs = np.reshape(self._inputs, (self._size, self._sequence_length, 1))

    def _normalize_inputs(self):
        self._inputs = self._inputs / float(self.num_classes)

    def _generate_random_data(self):
        for i in range(self._size):
            start = np.random.randint(self.num_classes - 2)
            end = np.random.randint(start, min(start + self._sequence_length, self.num_classes - 1))
            input_seq = alphabet[start:end + 1]
            output_seq = alphabet[end + 1]

            if(self._print_data):
                print("{}->{}".format(input_seq, output_seq))

            sample = self._encoding.encode_sequence(input_seq)
            label = self._encoding.encode_letter(output_seq)

            self._inputs.append(sample)
            self._labels.append(label)

def build_graph_writer(graph, logs_path='/tmp/tensorflow-workshop/alphabet-predictor'):
    current_time = datetime.datetime.now().strftime('%Y-%m-%d-%H%M')
    logs_path = path.join(logs_path, current_time)
    writer = tf.summary.FileWriter(logs_path, graph)
    return writer


def build_LSTM(x, sequence_length, num_units=32):
    with tf.name_scope("LSTM"):
        W = tf.Variable(tf.random_normal([num_units, num_units]), name="W")
        b = tf.Variable(tf.random_normal([1, num_units]), name="b")

        x = tf.split(x, sequence_length)
        inner_cells = [rnn.BasicLSTMCell(num_units=num_units) for _ in range(sequence_length)]
        rnn_cell = rnn.MultiRNNCell(inner_cells)
        outputs, states = rnn.static_rnn(rnn_cell, x, dtype=tf.float32)
        output = tf.nn.embedding_lookup(outputs, sequence_length - 1)
        output = tf.reshape(output, [1, num_units])

        tf.summary.histogram('weights', W)
        tf.summary.histogram('biases', b)

        return tf.matmul(output, W) + b

def build_fully_connected(x, num_units):
    with tf.name_scope("dense"):
        return tf.layers.dense(
            inputs=x,
            units=num_units)

dataset = Dataset(size=1000, print_data=False)
dataset.initialize()
e = Encoding()

x = tf.placeholder('float32', shape=(dataset.sequence_length, 1), name='inputs')
y = tf.placeholder('float32', shape=(1, len(alphabet)), name="labels")

lstm = build_LSTM(x, dataset.sequence_length)
dense = build_fully_connected(lstm, num_units=len(alphabet))

with tf.name_scope("loss"):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=dense, labels=y))
    tf.summary.scalar("loss", cost)

with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer().minimize(cost)

with tf.name_scope("accuracy"):
    pred = tf.argmax(dense, 1)
    accuracy, accuracy_update = tf.metrics.accuracy(labels=tf.argmax(y, 1), predictions=pred, name='acc')
    tf.summary.scalar("accuracy", accuracy)

merged = tf.summary.merge_all()
init = tf.global_variables_initializer()
init_locals = tf.local_variables_initializer()
with tf.Session() as session:
    session.run(init)
    session.run(init_locals)
    writer = build_graph_writer(session.graph)
    epoch = 0
    while epoch < 50:
        print("Training epoch {:<4d}".format(epoch), end='\t')
        dataset.shuffle()
        # Train the model
        for instance, label in zip(dataset.inputs, dataset.labels):
            session.run(optimizer, feed_dict={x: instance,
                                              y: label})
        # Calculate accuracy and loss
        for instance, label in zip(dataset.inputs, dataset.labels):
            summaries, acc, update_op, loss = session.run([merged, accuracy, accuracy_update, cost],
                                                          feed_dict={x: instance,
                                                                     y: label})
        writer.add_summary(summaries, epoch)
        print("Accuracy: {:.6f} \tLoss: {:.6f}".format(acc, loss))
        epoch = epoch + 1
    writer.close()

    seq = input('Enter a sequence of max 5 consecutive letters:')
    seq = seq.upper()
    print("You entered {}".format(seq))

    seq = e.encode_sequence(seq)
    seq = pad_sequences([seq], dataset.sequence_length)
    seq = np.reshape(seq, (dataset.sequence_length, 1))
    seq = seq / float(dataset.num_classes)

    result = session.run(pred, feed_dict={x: seq})
    letter = result[0]
    print("The next letter is: {}".format(e.decode_letter(letter)))
    session.close()